In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
model = "gstock-2b-test"
tickers = [ 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-3a-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [11]:
findata.EPOCHS=200
model = "cry-3a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-1a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-1b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
9,WEAT,5.01,0.81,27.14,34.92,5.14,5.680000,0.11,0.53,9.76,6.47,5.430000,0.06,0.61,12.73,10.82,4.960000,-0.04,0.06,4.933333
4,SOYB,13.68,0.84,87.39,55.81,24.97,23.030001,-0.08,0.65,44.31,9.25,25.330000,0.01,0.61,66.42,24.14,23.870001,-0.04,-0.04,4.400000
3,SLV,11.95,0.84,79.33,77.36,22.23,21.350000,-0.04,0.55,28.87,-14.88,23.500000,0.06,0.67,58.76,35.74,21.520000,-0.03,-0.03,3.066667
0,GLD,142.69,0.64,225.99,157.95,201.63,191.839996,-0.05,0.53,-0.41,45.41,200.850006,-0.00,0.62,146.20,118.54,193.449997,-0.04,-0.04,0.000000
1,ISHG,63.52,0.89,77.49,104.52,71.19,70.889999,-0.00,0.45,-24.88,-13.44,72.059998,0.01,0.63,27.06,28.65,70.760002,-0.01,-0.00,0.000000
2,PDBC,6.35,0.89,47.39,34.14,13.39,13.450000,0.00,0.62,19.10,9.75,14.020000,0.05,0.65,30.26,24.05,12.750000,-0.05,0.00,0.000000
5,TIP,96.93,0.70,68.77,59.24,107.59,112.199997,0.04,0.56,16.86,28.40,117.730003,0.09,0.64,49.08,49.87,106.900002,-0.01,0.04,0.000000
6,TLT,83.71,0.75,116.90,335.88,95.73,93.750000,-0.02,0.43,-122.18,44.49,105.680000,0.10,0.66,42.50,255.60,97.120003,0.01,0.01,0.000000
7,UUP,22.57,0.88,38.18,20.81,27.78,27.620001,-0.01,0.58,18.83,1.80,28.100000,0.01,0.54,12.55,6.77,27.420000,-0.01,-0.01,0.000000
8,VTIP,41.22,0.78,26.45,12.88,47.84,46.730000,-0.02,0.64,12.94,1.16,47.779999,-0.00,0.40,7.64,-6.40,46.900002,-0.02,-0.02,0.000000


In [ ]:
findata.EPOCHS=200
model = "vols-2b-test"
pending = ['ASTR', 'SMWB']
next = []
tickers = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LFMD', 'LUMN', 'MYTE', 'MAX', 'MTTR',
            'NEPH', 'NNOX', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SOUN', 'SST', 'STEM', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-2a-test"
next = ['AYX', 'FIVE']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-1b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [7]:
findata.EPOCHS=200
model = "com-2a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [13]:
addAllocHL(df, 0.10, 1)
# df1 = df[df.Gain > 0]
# df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
9,MATIC-USD,0.40,0.92,16.88,16.34,1.14,0.980000,-0.15,0.61,5.73,10.64,1.240000,0.08,0.78,14.75,13.75,0.960000,-0.16,-0.15,8.666667
2,BTC-USD,16794.05,0.88,275495.58,219225.51,68360.79,43847.480469,-0.36,0.70,255183.58,163048.00,64313.398438,-0.06,0.72,339503.38,187568.48,45945.589844,-0.33,-0.33,8.436364
5,ETH-USD,1074.15,0.87,27019.95,18565.03,3917.04,3386.860107,-0.14,0.65,13573.75,15496.04,3820.929932,-0.02,0.78,21315.57,23040.55,2718.219971,-0.31,-0.14,7.771429
13,XRP-USD,0.33,0.89,5.99,7.52,0.62,0.670000,0.08,0.56,4.44,2.88,0.720000,0.15,0.78,4.98,6.54,0.570000,-0.08,0.08,7.525000
1,AVAX-USD,10.90,0.86,600.73,674.02,43.12,47.320000,0.10,0.67,617.29,375.09,51.660000,0.20,0.70,444.26,413.04,39.000000,-0.10,0.10,7.200000
7,LINK-USD,5.71,0.88,133.36,168.67,20.30,18.840000,-0.07,0.70,124.77,120.52,22.410000,0.10,0.69,114.62,122.92,17.969999,-0.11,-0.07,7.085714
3,DOGE-USD,0.06,0.77,1.09,1.44,0.17,0.120000,-0.28,0.55,0.78,1.05,0.200000,0.17,0.72,0.91,1.53,0.140000,-0.19,-0.19,6.489474
6,FIL-USD,4.09,0.80,148.60,324.51,11.12,9.940000,-0.11,0.41,-60.39,175.82,12.760000,0.15,0.70,96.66,407.06,8.910000,-0.20,-0.11,6.181818
12,TRX-USD,0.05,0.81,0.55,0.48,0.14,0.120000,-0.15,0.73,0.70,0.36,0.130000,-0.06,0.58,0.48,0.22,0.130000,-0.06,-0.06,4.933333
8,LTC-USD,47.72,0.78,751.59,845.16,89.44,95.330002,0.07,0.61,449.87,494.03,103.089996,0.15,0.66,702.61,1098.88,82.980003,-0.07,0.07,4.657143


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [14]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
cry-3a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
cry-3a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
cry-3a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,649,239 bytes  received 73 bytes  19,298,624.00 bytes/sec
total size is 215,232,944  speedup is 22.31
